<a href="https://colab.research.google.com/github/idhamari/LapIRN/blob/master/demoLapIRN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3D Medical Image Registration with DNN

Using Laplacian Pyramid Deep Learning Networks (LAPIRN)

This is a demo notebook for a github repository [LabIRN](https://github.com/cwmok/LapIRN/blob/master/Code/Train_LapIRN_disp.py). The original code author is [Tony Chi Wing MOK](https://cwmok.github.io/). If you have questions please open issue in the original repository. 

the paper can be downloaded from [here](https://arxiv.org/abs/2006.16148).


The dataset used is from [learn2reg challenge](https://learn2reg.grand-challenge.org/Datasets). It can be downloaded from ()

Notes:
  - some bugs are fixed.

Todos:
  - use tensorboard to draw training and testing curves 
  - improve the reading/writing procedures
  - remove redundant code


**This notebook is controbuted by:** Ibraheem Al-Dhamari
  


# LapIRN: Large Deformation Diffeomorphic Image Registration with Laplacian Pyramid Networks 



## Introduction

The goal of **deformable registration** is computing a transformation to align a pair of images by minimizing a similarity metric that maximizes the similarity between the images. Usually, we call these images the fixed (or reference) image and the moving (or target) image. 

Deformable registration has many important applications and considered one of the most challenging problems. The general image registration problem is not solved despite many published papers every year trying to solve a special case of this general registration problem.  

The **multi-resolution strategy** estimates the target transformation iteratively along number of levels. Each level estimates a transformation of a higher level. This usually produces faster computation and better results. 

One disadvantage of the conventional image registration methods is the large time required to complete the task. **Deep learning networks (DNN)** requires much less time to accomplish such task. 

In DNN, the image registration problem can be represented as pixel-wise image translation problem. The network learns the pixel-wise spatial correspondence of a pair of images using convolution. 




## Method

Given a fixed 3D image $F$ and a moving 3D image $M$, the learning function $f$ computes a deformation field $\phi$ using the learning parameters $\theta$: 

$$
 f_{\theta}(F,M,\phi_{L-1},\theta_{L-1})=\phi
$$

This will be repeated for number of levels e.g. for three levels of multi-resolution $L=3$:

$$
 f_{\theta_3}(F_3,M_3)=\phi_3 \\
 f_{\theta_2}(F_2,M_2,\phi_3,\theta_3)=\phi_2 \\
 f_{\theta}(F,M,\phi_2,\theta_2)=\phi \\
$$

In general:

$$
 f_{\theta_L}(F_L,M_L,\phi_{L-1},\theta_{L-1})=\phi_L \\
$$

where the size of $F_L$ is the size of F / ${2^{L-1}}$. In other words: we get as maller size image by half at each level. In the final level, we use the original image size.




## Architecture


a L-level Laplacian pyramid framework to mimic the conventional multi-resolution strategy. For simplicity, we set L to 3 throughout this paper. The overview of LapIRN is illustrated in Fig. 1. Specifically, we first create the input image pyramid by downsampling the input images with trilinear interpolation to obtain Fi ∈ {F1, F2, F3} (and Mi ∈ {M1, M2, M3}), where Fi denotes the downsampled F with a scale factor 0.5(L−i) and F3 = F. We employ a CNN-based registration network (CRN) to

# Setup

In [ ]:
# It is better to work with google drive as downloading is not working!

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

installTools    = 1 # every time you restart colab virtual machine
cloneGitRep     = 0 # one time thing, it will be saved in google drive
downloadDataset = 0 # one time thing, it will be saved in google drive
if installTools:
    !pip3 install -U scikit-learn
    !pip3 install simpleitk
    !pip3 install tensorflow==1.14 tensorflow-gpu==1.14 keras==2.3.1
    !pip3 install nibabel tqdm
    !pip3 install torchvision torch==1.3.0 
    print("installing tools is done!")

if cloneGitRep :   
    #clone the updated code
    !ls
    wdPath = 'drive/MyDrive/LapIRN_org'
    !mkdir $wdPath
    !ls drive/MyDrive
    !git clone https://github.com/idhamari/LapIRN.git $wdPath/LapIRN
    print('-------------------')
    !ls  $wdPath/LapIRN
    print("cloning repository is done!")

if downloadDataset:    
    # download the dataset    
    !mkdir $wdPath/datasets
    # note use the same lines with changing the only the id
    !curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=17uysjRAiXMIT2QApW5kHWP1aHCi5_lPO" > tmp.txt
    !curl -L -b /tmp/cookies "https://drive.google.com$(cat tmp.txt | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" >  $wdPath/datasets/L2R_Task3_AbdominalCT_160x192x144.zip
    !ls  $wdPath/datasets/L2R_Task3_AbdominalCT_160x192x144.zip -l --block-size=M 
    !unzip $wdPath/datasets/L2R_Task3_AbdominalCT_160x192x144.zip -d $wdPath/datasets 
    !ls   $wdPath/datasets
    print("downloading dataset is done!")
    firstRun = 0

!ls

import time , shutil, os
from google.colab import files
%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # first gpu

# print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
# print("      important note: copy or download the results if you want to save them")
# print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")

     |████████████████████████████████| 22.3MB 1.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 47.4MB 64kB/s 
     |████████████████████████████████| 109.3MB 38kB/s 
     |████████████████████████████████| 377.1MB 44kB/s 
     |████████████████████████████████| 378kB 40.2MB/s 
     |████████████████████████████████| 491kB 37.1MB/s 
     |████████████████████████████████| 3.2MB 35.6MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully 

# Training

In [ ]:
doTraining = 1
if doTraining:
    import time , shutil, os
    from google.colab import files
    %matplotlib inline

    !ls
    # work_directory
    wdPath = 'drive/MyDrive/LapIRN_org/'
    # # dataset
    dataset_path = '../../datasets/L2R_Task3_AbdominalCT_160x192x144'
    # # 30 images,  image size =  192,160,256, each has its segmentation, the segmentation has 13 segmentation classes,
    # images are already resized to fit the GPU memory 

    # remove old training files:

    scriptPath = wdPath + 'LapIRN/demoLapIRN_org.py'
    doTrain     = " 1 "
    isLocal     = " 0 " # use gitlab or local workstation
    slvl1       = " 3000 "  # start epoch lvl1    
    slvl2       = " 3000 "  # start epoch lvl2       
    slvl3       = " 0 "  # start epoch lvl3     
    lvl1        = " 3000 "   # number of iterations for first resolution  
    lvl2        = " 3000 "  # number of iterations for second resolution  
    lvl3        = " 30000 " # number of iterations for third resolution 
    checkpoint  = " 10 "# model will be saved after each checkpoint steps    

    if not ( (int(slvl1)>0) or (int(slvl2)>0) or (int(slvl3)>0)) :
       if os.path.isdir(wdPath + 'LapIRN/Model/Stage'):
           print("removing old stage files .......................................")
           shutil.rmtree(wdPath + 'LapIRN/Model/Stage')
       else: 
          print("folder not found ..... "+wdPath + 'LapIRN/Model/Stage')    


    wdPath = '/content/drive/MyDrive/LapIRN_org/'
    #!python $scriptPath $doTrain $slvl1 $slvl2 $slvl3 $lvl1 $lvl2 $lvl3 $checkpoint $wd_path $dataset_path
    # cmd = "python " + scriptPath + " " + doTrain + isLocal + slvl1 +slvl2 +slvl3 +lvl1 +lvl2 +lvl3 + checkpoint  + " " +wdPath  + " " +dataset_path
    # os.system(cmd)
    
    !python $scriptPath $doTrain $isLocal $slvl1 $slvl2 $slvl3 $lvl1 $lvl2 $lvl3 $checkpoint  $wdPath  $dataset_path

    #save data every one hour
    # print("process started ....")
    # while 1:
    #     # save results every some hours
    #     if os.path.isfile('model_folder_compressed.zip'):
    #         os.remove('model_folder_compressed.zip')
    #     print("creating archive ...............")         
    #     time.sleep(60*60)
    #     shutil.make_archive('model_folder_compressed', 'zip', '/content/LapIRN_org/LapIRN/Model')      
    #     print("downloading archive ...............")         
    #     time.sleep(15*60)
    #     files.download('model_folder_compressed.zip') 
    #     time.sleep(15*300)
    #     
    # # download the pretrained model
    print(" training done! ....................")

drive  sample_data
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
                 Setup                                        
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be un

# Testing

In [ ]:
#run testing on testing dataset and save results 
doTesting = 0
if doTesting:
    import time , shutil, os
    from google.colab import files
    %matplotlib inline

    # testing download a folder and a file 

    # !apt install nano
    # !nano myTxt14042021.txt
    #files.upload()  # or use the left side panel 
    !ls
    # work_directory
    wdPath = 'drive/MyDrive/LapIRN_org/'
    # # dataset
    dataset_path = '../../datasets/L2R_Task3_AbdominalCT_160x192x144'    # # dataset
    # # 30 images,  image size =  192,160,256, each has its segmentation, the segmentation has 13 segmentation classes,
    # images are already resized to fit the GPU memory 

    scriptPath = wd_path + 'LapIRN/demoLapIRN_org.py'
    doTrain     = " 0 "
    isLocal     = " 0 " # use gitlab or local workstation
    slvl1       = " 0 "  # start epoch lvl1    
    slvl2       = " 0 "  # start epoch lvl2       
    slvl3       = " 0 "  # start epoch lvl3     
    lvl1        = " 3000 "   # number of iterations for first resolution  
    lvl2        = " 3000 "  # number of iterations for second resolution  
    lvl3        = " 30000 " # number of iterations for third resolution 
    checkpoint  = " 500 "# model will be saved after each checkpoint steps


    #!python $scriptPath $doTrain $slvl1 $slvl2 $slvl3 $lvl1 $lvl2 $lvl3 $checkpoint $wd_path $dataset_path
    cmd = "python " + scriptPath + " " + doTrain + isLocal + slvl1 +slvl2 +slvl3 +lvl1 +lvl2 +lvl3 +checkpoint  + " " +wd_path  + " " +dataset_path
    os.system(cmd)

     
    # # download the pretrained model
    print("done! ....................")



# Analysis

In [ ]:
#Draw results 